In [2]:
import torch
import torch.nn as nn
import numpy as np
import logging
from torch.utils.data import Dataset, DataLoader
from utils import normalize, cleanTrumpData, Dataset_trumpOld, Dataset_trump, strs2code 
from textloader import TextDataset, collate_fn
from tp5 import RNN, GRU, LSTM
import string

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.basicConfig(level=logging.INFO)

In [3]:
PAD_IX = 0
EOS_IX = 1

#LETTRES = string.ascii_letters + string.punctuation+string.digits+' '
#LETTRES = string.ascii_letters+' '
LETTRES = string.ascii_letters[:26]+"."+' '
id2lettre = dict(zip(range(2, len(LETTRES)+2), LETTRES))
id2lettre[PAD_IX] = '' ##NULL CHARACTER
id2lettre[EOS_IX] = '|'
lettre2id = dict(zip(id2lettre.values(), id2lettre.keys()))

In [4]:
with open("data/trump_full_speech.txt", 'r') as f:
    data = f.read()

In [7]:
cleanedNormalizedData = normalize(cleanTrumpData(data).lower().replace("!",".").replace("?","."))

In [8]:
BATCH_SIZE = 3
loader = DataLoader(TextDataset(cleanedNormalizedData), collate_fn=collate_fn, batch_size=BATCH_SIZE)

# Classification de sequence

# Generation de sequence

# Test crossEntropy with batch 

In [23]:
loss = nn.CrossEntropyLoss(reduction="none")
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [25]:
input

tensor([[ 0.6207, -0.7542,  0.0652,  0.3838, -3.1213],
        [ 0.3055, -1.0484,  0.3335,  0.2342, -0.8832],
        [ 0.5134, -0.6117,  0.8091,  0.4683,  0.0508]], requires_grad=True)

In [26]:
target

tensor([4, 0, 2])

In [24]:
output

tensor([4.7126, 1.2592, 1.1522], grad_fn=<NllLossBackward>)

In [27]:
loss2 = nn.CrossEntropyLoss(reduction="mean")

In [28]:
output2 = loss2(input, target)

In [29]:
output2

tensor(2.3747, grad_fn=<NllLossBackward>)

In [30]:
output.mean()

tensor(2.3747, grad_fn=<MeanBackward0>)